# Converting between yml and xlsx meta-data formats

**Author: Gao Wang**

This notebook contains a pipeline to convert between xlsx and yaml formats for phenotype meta data.

Featuers will be implemented on need basis (as our thoughts on meta-data evolves) and will be documented in the section below.

## Example usage

In [1]:
sos run xlsx_to_yml.ipynb --xlsx-file FGC_metadata_v4_template.xlsx

INFO: Running default: 
INFO: default is completed.
INFO: default output:   FGC_metadata_v4_template.yml
INFO: Workflow default (ID=w9997ae7f947c9a21) is executed successfully with 1 completed step.


## Implementation details

In [ ]:
[global]
parameter: xlsx_file = str
    
[default]
depends: Py_Module("openpyxl"), Py_Module("numpyencoder")
input: xlsx_file
output: f"{_input:n}.yml"
python: expand = "${ }"
    import pandas as pd
    df = pd.read_excel(${_input:r}, sheet_name=None, dtype=str, engine="openpyxl")
    values = dict([(sheet, dict([(x.split(":")[1].strip(), df[sheet][x][0]) for x in df[sheet].columns]))
              for sheet in df])
    # need to dump to plain JSON first in order to properly YAML format it
    import json
    from numpyencoder import NumpyEncoder
    json.dump(values, open(${_output:r}, "w"), indent=4, sort_keys=False,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)
    import yaml
    values = json.load(open(${_output:r}))
    yaml.safe_dump(values, open(${_output:r}, "w"), 
               allow_unicode=True, 
               default_flow_style=False, 
               indent=4, 
               sort_keys=False)